In [3]:
import pandas as pd

#<font color='orange'> **Data preprocessing**

In [10]:
df = pd.read_csv('region_04.csv')
df_clean = df.dropna(subset=['latitude', 'longitude'])
columns_to_keep = ['eventid', 'iyear', 'imonth', 'iday', 'country_txt', 'region_txt', 'city', 'latitude', 'longitude', 'attacktype1_txt', 'nkill', 'nwound']
df_clean = df_clean[columns_to_keep]

# <font color='orange'>**Stacked area chart using plotly**

In [11]:
import plotly.graph_objects as go

attack_counts = df_clean.groupby(['iyear', 'attacktype1_txt']).size().unstack(fill_value=0)

# Traces for each attack type
fig = go.Figure()
for attack_type in attack_counts.columns:
    fig.add_trace(go.Scatter(
        x=attack_counts.index,
        y=attack_counts[attack_type],
        mode='lines',
        stackgroup='one',
        name=attack_type,
        hovertemplate='<b>Year:</b> %{x}<br><b>Attack Type:</b> ' + attack_type + '<br><b>Count:</b> %{y}'
    ))

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {"args": [{"y": [attack_counts[col] for col in attack_counts.columns], "stackgroup": "one"}], "label": "Stacked", "method": "update"},
                {"args": [{"stackgroup": ""}], "label": "Grouped", "method": "restyle"},
                {"args": [{"y": [attack_counts[col] / attack_counts.sum(axis=1) for col in attack_counts.columns], "stackgroup": "one"}],
                 "label": "100% Stacked", "method": "update"}
            ],
            "direction": "down",
            "showactive": True,
            "x": 1.15,
            "xanchor": "right",
            "y": 1.15,
            "yanchor": "top"
        }
    ],
    title="Attack Types Over the years",
    xaxis_title="Year",
    yaxis_title="Number of Incidents",
    legend_title="Attack Types"
)

fig.show()

# <font color='orange'>**Interactive Scatter plot using Bokeh**

In [9]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Slider, CustomJS, CategoricalColorMapper, Legend, LegendItem
from bokeh.layouts import column
from bokeh.io import output_file
from bokeh.palettes import Category20

# Filling missing values with 0 to avoid bias
df = pd.read_csv('region_04.csv')
df.fillna({'nkill': 0, 'nwound': 0}, inplace=True)
df['total_casualties'] = df['nkill'] + df['nwound']

unique_targets = df['targtype1_txt'].unique()
color_palette = Category20[len(unique_targets)] if len(unique_targets) <= 20 else Category20[20]
color_map = dict(zip(unique_targets, color_palette))

df['color'] = df['targtype1_txt'].map(color_map)

source = ColumnDataSource(df)

# Scatter plot
p = figure(
    title="Target Types and Casualties",
    x_axis_label="Number of Killed",
    y_axis_label="Number of Wounded",
    tools="pan,wheel_zoom,box_zoom,reset",
)

scatter = p.scatter(
    x='nkill', y='nwound', source=source,
    size='total_casualties', fill_alpha=0.6,
    line_color="black", color='color', legend_field="targtype1_txt"
)

# Hover tool
p.add_tools(HoverTool(tooltips=[
    ("Target", "@targtype1_txt"),
    ("Killed", "@nkill"),
    ("Wounded", "@nwound"),
    ("Total Casualties", "@total_casualties")
]))

# slider for filtering by year
year_slider = Slider(start=df['iyear'].min(), end=df['iyear'].max(), value=df['iyear'].min(), step=1, title="Year")

# Convert DataFrame to a dictionary format for JavaScript
df_dict = df.to_dict(orient="list")

# JavaScript callback for filtering
callback = CustomJS(args=dict(source=source, df_dict=df_dict, slider=year_slider), code="""
    var data = source.data;
    var all_data = df_dict;
    var selected_year = slider.value;

    var nkill = [];
    var nwound = [];
    var total_casualties = [];
    var targtype1_txt = [];
    var color = [];

    for (var i = 0; i < all_data['iyear'].length; i++) {
        if (all_data['iyear'][i] == selected_year) {
            nkill.push(all_data["nkill"][i]);
            nwound.push(all_data["nwound"][i]);
            total_casualties.push(all_data["total_casualties"][i]);
            targtype1_txt.push(all_data["targtype1_txt"][i]);
            color.push(all_data["color"][i]);
        }
    }

    data["nkill"] = nkill;
    data["nwound"] = nwound;
    data["total_casualties"] = total_casualties;
    data["targtype1_txt"] = targtype1_txt;
    data["color"] = color;

    source.change.emit();
""")

year_slider.js_on_change('value', callback)

# saving as html file
layout = column(year_slider, p)
output_file("scatter_plot.html")
show(layout)